# Demystifying Gaussian Process Regression

#### By Brett Morris, Jessica Birky, Russell Van Linge

### In this tutorial

The purpose of this tutorial is to introduce the concepts of Gaussian process regression, and give users an interactive environment for experimenting with Gaussian processes. This tutorial is based on [Rasmussen and Williams (2006)](http://www.gaussianprocess.org/gpml/), but is designed to be more readable and to encourage experimentation!


### Getting started


Let's generate some fake, one dimensional data, which we'll fit with various techniques.

First we need to import matplotlib, numpy, (and optionally scipy) for this notebook: 

In [ ]:
%matplotlib inline
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt

Now let's generate a small dataset of observations $y$, observed at times $x$:

In [ ]:
np.random.seed(42)
ndim = 10
y = np.random.rand(ndim)[:, np.newaxis]
y -= y.mean()
x = np.arange(len(y))[:, np.newaxis]
yerr = y.std() / 2 * np.ones_like(x)

plt.figure(figsize=(8, 6))
plt.errorbar(x, y, yerr, fmt='.', color='k')
plt.xlabel('x')
plt.ylabel('y');

***

# Linear regression

Solutions to the [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) estimators $\hat{\beta}$ are

$$ \hat{\beta} = ({\bf X}^\top {\bf N}^{-1} {\bf X})^{-1} {\bf X}^\top {\bf N}^{-1} y,$$

and their uncertainties are given by

$$ \mathrm{cov} = {\bf X}^\top {\bf N}^{-1} {\bf X}, $$

where ${\bf N}$ is the matrix of uncertainties on measurements $y$ (${\bf N}$ is diagonal for measurements with independent Gaussian uncertainties, in this case). 

Let's implement this using numpy! Note: the `@` operator can be used in python>3.5 to multiply matrices together.

In [ ]:
# Append a column of ones next to the `x` values using `np.vander`: 
X = np.vander(x.ravel(), 2)
inv_N = np.linalg.inv(np.identity(len(x)) * yerr**2)

# Solve linear regression: 
betas = np.linalg.inv(X.T @ inv_N @ X) @ X.T @ inv_N @ y
cov = np.linalg.inv(X.T @ inv_N @ X)

# Compute best fit line: 
best_fit = X @ betas 
err = np.sqrt(np.diag(cov))

Plot the result and its uncertainty (here we're plotting just the uncertainty in the intercept and ignoring the uncertainty in the slope, for this example):

In [ ]:
plt.figure(figsize=(8, 6))
plt.errorbar(x, y, yerr, fmt='.', color='k')
plt.plot(x, best_fit)
plt.fill_between(x.ravel(), best_fit.ravel()-err[1], best_fit.ravel()+err[1], alpha=0.3)
plt.xlabel('x')
plt.ylabel('y');

*** 
# Gaussian process regression

Gaussian process regression, in this example, is a technique for interpolating between observations or predicting the values of observations at a given time. The linear algebra this time looks a little more complicated, but it's a similar process:   

We define a "**kernel function**"

$$ {\bf K}({\bf x_1}, {\bf x_2}) = \exp\left(-\frac{(x_1 - x_2)^2)}{2 \ell^2}\right)   $$

A **kernel function** describes the correlation between neighboring points. To start, we're using "squared exponential" kernel, which equivalent to saying "the correlation between neighboring points is defined by a Gaussian with one parameter: $\ell$." We call the tunable parameters within the kernel function **hyperparameters**. Don't let the fancy name intimidate you – all hyperparameters do is define the kernel function, which describes how correlated neighboring points are. 

##### The $\ell$ hyperparameter 
The $\ell$ parameter sets the autocorrelation timescale, in other words, how far two points need to be from each other in $x$ (time) before they are no longer correlated. If $\ell$ is large, distant points become more correlated, or in other words, the function becomes smoother. If $\ell$ is small, the function varies more rapidly with $x$. 


### Predicting observations at arbitrary $x$
Let's say that we've taken observations at times ${\bf x}$ and we'd like to predict value of $y$ at new times ${\bf x_\star}$. The predicted mean is (Equation 2.25 of [Rasmussen and Williams (2006)](http://www.gaussianprocess.org/gpml/)) 

$$ \mu_* = {\bf K(x, x_*)}^\top ({\bf K(x, x)} + \sigma_n^2 {\bf I})^{-1} {\bf y},  $$

and the covariance is (Equation 2.26)

$$ \mathrm{cov} = {\bf K}({\bf x_*}, {\bf x_*})- {\bf K(x, x_*)}^\top ({\bf K(x, x)}+ \sigma_n^2 {\bf I})^{-1} {\bf K(x, x_*)}, $$ 

and therefore the error on the predictions will be

$$ \mathrm{err} = \mathrm{diag}\left( \sqrt{\mathrm{cov}} \right).$$

We can implement this in just a few lines of Python:

In [ ]:
def square_distance(x1, x2): 
    """
    Compute the squared distance between two vectors `x1` and `x2` 
    
    Note that (x1 - x2)^2 = x1^2 + x2^2 - 2 * x1 * x2, so we can use the short syntax below
    to avoid loops: 
    
    For reference, see: 
    https://medium.com/dataholiks-distillery/l2-distance-matrix-vectorization-trick-26aa3247ac6c
    """
    return np.sum(x1**2, axis=1)[:, np.newaxis] + np.sum(x2**2, axis=1) - 2 * x1 @ x2.T

def sq_exp_kernel(x1, x2, ell=1): 
    """
    Gaussian Kernel function
    """
    sqdist = square_distance(x1, x2)

    return np.exp(-0.5 * sqdist / ell**2)

def gaussian_process_regression(x, y, yerr, xtest, kernel): 
    """
    Gaussian process regression for column vectors `x` and `y` with 
    uncertainties `yerr`; predict values at `xtest` using `kernel`
    """
    K = kernel(x, x) + np.identity(len(x)) * yerr**2
    k_s = kernel(x, xtest)
    k_ss = kernel(xtest, xtest)
    inv_K = np.linalg.inv(K)
    # The `@` operator in python 3 is the matrix multiplication operator
    mu = k_s.T @ inv_K @ y
    cov = k_ss - k_s.T @ inv_K @ k_s
    return mu, cov

Let's call our Gaussian process regression method on our data, interpolating between the observations: 

In [ ]:
N = 100
# Interpolate `N` observations from the minimum to maximum values of `x` 
xtest = np.linspace(x.min(), x.max(), N)[:, np.newaxis]

mu, cov = gaussian_process_regression(x, y, yerr, xtest, sq_exp_kernel)

err = np.sqrt(np.diagonal(cov))

plt.figure(figsize=(8, 6))
plt.errorbar(x, y, yerr, fmt='.', color='k')
plt.plot(xtest.ravel(), mu.ravel(), label='GP mean')
plt.fill_between(xtest.ravel(), mu.ravel()-err, mu.ravel()+err, alpha=0.3, label='GP uncertainty')
plt.legend(loc='upper center')
plt.xlabel('x')
plt.ylabel('y');

In the figure above, the blue line shows the predicted value of $y$ for values of $x$ where there were no observations – that's why we say Gaussian process regression is a form of interpolation. It's essentially interpolating between points, where the smoothness of the interpolation is set by the kernel function. 

But Gaussian process regression is more powerful than just predicting the value of $y$ for arbitrary $x$, it also computes the uncertainty of prediction for arbitrary $x$. The blue region in the figure above shows the uncertainty on the predicted value of $y$ at each $x$. Note that the uncertainty is equivalent to the width of the error bars where there are observations, and the uncertainty gets larger in between observations. 

Gaussian process regression is often referred to as a form of **"non-parametric"** parameter estimation, which is a way of saying that you don't define the functional form of the mean, $\mu$. $\mu$ is not a polynomial function, it is not a trigonometric function, it is a highly adaptive interpolation function whose functional form is in fact defined by its covariance matrix (${\bf K}$), rather than being some function $\mu = f(x)$ (which means typically it's not easy to write the functional form of the GP in terms of $x$).

### Square-Exponential (Gaussian) Kernel

Let's fit the data with an interactive kernel, which allows you to vary the $\ell$ hyperparameter and see the results: 

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def gp_interact(ell):
    def sq_exp_kernel_interactive(x1, x2=None, ell=ell): 
        """
        Interactive Gaussian Kernel function
        """
        if x2 is not None: 
            sqdist = square_distance(x1, x2)
        else: 
            sqdist = x1**2
            
        return np.exp(-0.5 * sqdist / ell**2)
    
    mu, cov = gaussian_process_regression(x, y, yerr, xtest, sq_exp_kernel_interactive)
    err = np.sqrt(np.diag(cov))
    
    fig, ax = plt.subplots(1, 3, figsize=(14, 4))
    ax[0].errorbar(x, y, yerr, fmt='.', color='k')
    ax[0].plot(xtest.ravel(), mu.ravel(), label='GP Mean')
    ax[0].fill_between(xtest.ravel(),  mu.ravel()-err, mu.ravel()+err, alpha=0.2, label='GP Uncertainty')
    ax[0].set(xlabel='$x$', ylabel='$y$', title='GP Regression')
    ax[0].legend(loc='upper center')
    
    ax[1].set(xlabel='$x_1$', ylabel='$x_2$', title='Covariance matrix')
    ax[1].imshow(sq_exp_kernel_interactive(x, x))
    
    ax[2].plot(xtest, sq_exp_kernel_interactive(xtest))
    ax[2].set(xlabel='Lags', ylabel='Autocorrelation', title='Autocorrelation Function')
    fig.tight_layout()
    plt.show()

print('Vary the hyperparameter "ell", which defines the autocorrelation timescale:')
interactive_plot = interactive(gp_interact, ell=(1, 10, 1))
output = interactive_plot.children[-1]
interactive_plot

In the figure above, the left plot shows the mean model (blue) with its uncertainty (blue shaded region), and the data (black circles). 

The middle plot shows the covariance matrix – note that it's got most of its weight along the diagonal, which is a mathematical representation of saying "observations close in $x$ to one another are more correlated than observations far apart in $x$". The brightness of each pixel corresponds to the strength of the correlation between the two inputs $x_1$ and $x_2$. 

Alternatively, a simpler way to visualize the behavior of your kernel function is to plot the kernel function directly, which is shown on the right. Note how distant points (large lags) have stronger autocorrelation power when `ell` is large.


### Exponential-cosine kernel 

The exponential-cosine kernel is a cosine function multiplied by a decaying exponential envelope. It is good at fitting quasi-periodic signals (like stellar photometry, for example). 

In [ ]:
def gp_interact(ell, period):
    def exp_cos(x1, x2=None, sigma=1, ell=ell, period=period): 
        """
        Exponentially-decaying cosine function
        """
        if x2 is not None:
            sqdist = square_distance(x1, x2)
        else: 
            sqdist = x1**2
            
        return np.exp(-0.5 * sqdist / ell**2) * np.cos(2*np.pi*np.sqrt(sqdist)/period)
    
    mu, cov = gaussian_process_regression(x, y, yerr, xtest, exp_cos)
    err = np.sqrt(np.diag(cov))
    
    fig, ax = plt.subplots(1, 3, figsize=(14, 4))
    ax[0].errorbar(x, y, yerr, fmt='.', color='k')
    ax[0].plot(xtest.ravel(), mu.ravel(), label='GP Mean')
    ax[0].fill_between(xtest.ravel(),  mu.ravel()-err, mu.ravel()+err, alpha=0.2, label='GP Uncertainty')
    ax[0].set(xlabel='$x$', ylabel='$y$', title='GP Regression')
    ax[0].legend(loc='upper center')
    
    ax[1].set(xlabel='$x_1$', ylabel='$x_2$', title='Covariance matrix')
    ax[1].imshow(exp_cos(x, x))
    
    ax[2].plot(xtest, exp_cos(xtest))
    ax[2].set(xlabel='Lags', ylabel='Autocorrelation', title='Autocorrelation Function')
    fig.tight_layout()
    plt.show()

print('Vary the hyperparameters "ell" and "period", which define the autocorrelation timescale:')
interactive_plot = interactive(gp_interact, ell=(1, 10, 1), period=(1, 10, 1))
output = interactive_plot.children[-1]
interactive_plot

It's easiest to see what this choice of kernel function (exp-cos) choice looks like by watching the autocorrelation plot above as you change the parameters.  

##### The Matrix Inversion Bottleneck

Now you may be asking yourself, "wow this is great, I'd like to apply this technique to a huge dataset. How does the GP scale with the size of the dataset?" The answer is a bit scary – the computation expense in the general case scales as $\mathcal{O}(N^3)$. The most computationally expensive step in the process here is the matrix inversion. We're using `numpy`'s standard matrix inversion function, which should work on any matrix.

If you want a faster implementation of your Gaussian process regression, you can use a package like [`george`](https://github.com/dfm/george) or [`celerite`](https://github.com/dfm/celerite) which implement faster matrix inversions by taking advantage of special properties of the covariance matrices of certain kernel functions. 

# Example: a rotating star

Gaussian processes are not just a means of marginalizing over noise in observations, they can also be used to *measure* physical quantities. Say for example that you are measuring the flux $f$ of a rotating star as a function of time $t$: 

In [ ]:
true_period = 3 # days

np.random.seed(42)
npoints = 30
t = np.linspace(0, 10, npoints)[:, np.newaxis]
f = np.cos(2*np.pi*t/true_period) + 0.5 * np.random.randn(len(t))[:, np.newaxis]
ferr = 0.5 * np.ones_like(t)

t_test = np.linspace(t.min(), t.max(), 100)[:, np.newaxis]

plt.errorbar(t, f, ferr, color='k', fmt='.')
plt.xlabel('Time [days]')
plt.ylabel('Flux');

There's a lot of noise in these observations, but there's also a signal – a periodic signal of rotation from the star. 

In [ ]:
def gp_interact(ell, period):
    def exp_cos(x1, x2=None, sigma=1, ell=ell, period=period): 
        """
        Exponentially-decaying cosine function
        """
        if x2 is not None:
            sqdist = square_distance(x1, x2)
        else: 
            sqdist = x1**2
            
        return np.exp(-0.5 * sqdist / ell**2) * np.cos(2*np.pi*np.sqrt(sqdist)/period)
    
    mu, cov = gaussian_process_regression(t, f, ferr, t, exp_cos)
    chi2 = np.sum((mu - f)**2/ferr**2)
    lnlike = -0.5 * chi2
    
    mu, cov = gaussian_process_regression(t, f, ferr, t_test, exp_cos)
    err = np.sqrt(np.diag(cov))
    
    fig, ax = plt.subplots(1, 3, figsize=(14, 4))
    ax[0].errorbar(t, f, ferr, fmt='.', color='k')
    ax[0].plot(t_test.ravel(), mu.ravel(), label='GP Mean')
    ax[0].fill_between(t_test.ravel(),  mu.ravel()-err, mu.ravel()+err, alpha=0.2, label='GP Uncertainty')
    ax[0].set(xlabel='Time [days]', ylabel='Flux', title='GP Regression')
    ax[0].legend(loc='upper center')
    
    ax[1].set(xlabel='$x_1$', ylabel='$x_2$', title='Covariance matrix')
    ax[1].imshow(exp_cos(t, t))
    
    ax[2].plot(xtest, exp_cos(xtest))
    ax[2].set(xlabel='Lags', ylabel='Autocorrelation', title='Autocorrelation Function')
    fig.tight_layout()
    plt.show()
    display("Log-likelihood: {0}".format(lnlike))


print('Vary the hyperparameters "ell", and "period" which define the autocorrelation timescale:')
interactive_plot = interactive(gp_interact, ell=(1, 10, 1), period=(1, 10, 0.5))
output = interactive_plot.children[-1]
interactive_plot

Now vary the `period` and `ell` parameters using the sliders – what parameters give you the best fit to the data? Do they match the input period (`true_period`)? You can refer to the log-likelihood, which is printed at the bottom of the visualizatiaon. Log-likelihood is just the opposite of $\chi^2$, you want to *maximize* the log-likelihood to find the best-fit parameters.

### Solving for the maximum-likelihood hyperparameters

We can use an optimizer to fit for the best period: 

In [ ]:
from scipy.optimize import minimize

def chi2(p):
    ell, period = p
    
    def exp_cos(x1, x2, sigma=1, ell=ell, period=period): 
        """
        Exponentially-decaying cosine function
        """
        sqdist = square_distance(x1, x2)

        return np.exp(-0.5 * sqdist / ell**2) * np.cos(2*np.pi*np.sqrt(sqdist)/period)
    
    mu, cov = gaussian_process_regression(t, f, ferr, t, exp_cos)
    chi2 = np.sum((mu - f)**2/ferr**2)
    return chi2

init_parameters = [1.5, 3.5]
solution = minimize(chi2, init_parameters)
best_ell, best_period = solution.x

print("Maximum-likelihood period: {0:.2f} days".format(best_period))

If the value above is close to the input period, good job, you've just used a quasi-periodic kernel function to find the period buried in noisy data! 


### Choosing a Kernel 

You should try whenever possible to choose a kernel that aligns with the source of your noise. For example, if your noise is smoothly varying, a squared-exponential kernel might do the trick. If your data is more rapidly varying, you may find a better fit with a Matern kernel. If you know that your data have a quasi-periodic trend (like rotating stars), fit with an exponential-cosine kernel. See Chapter 4 of [Rasmussen and Williams](http://www.gaussianprocess.org/gpml/) for the full range of possibilities.

### Overfitting

You must be careful to set a reasonable prior on the timescale parameter of your GP – if you don't, an optimizer will likely tweak your hyperparameters such that the timescale parameter is small, allowing high frequency variations in the time domain which fit every peak and trough in your data. A good place to start is to prevent the timescale parameters from being similar to or smaller than the space between observations. 


### Closing thoughts

* A Gaussian process (GP) is a non-parametric approach to modeling noisy data

* GPs are useful when you don't know the explicit functional form of the correlated noise in your data

* GPs are useful for measuring physical parameters in certain circumstances (e.g.: fits for a period in noisy quasi-periodic observations)

* GPs are slow in general, so you should take advantage of packages like [`celerite`](http://celerite.readthedocs.io) and [`george`](http://george.readthedocs.io) for fast matrix inversion/decomposition tricks to speed things up

* This tutorial only covers GPs in one dimension, though they are extensible to N dimensions (see for example: [GPyTorch](https://github.com/cornellius-gp/gpytorch))